In [1]:
from agent_framework import FileCheckpointStorage
from pathlib import Path

# Initialize checkpoint storage
checkpoint_storage = FileCheckpointStorage(storage_path="./checkpoints")

In [2]:
from agent_framework import WorkflowBuilder

workflow = (
    WorkflowBuilder(max_iterations=5)
    .add_edge(executor1, executor2)
    .set_start_executor(executor1)
    .with_checkpointing(checkpoint_storage=checkpoint_storage)  # Enable checkpointing
    .build()
)

NameError: name 'executor1' is not defined

In [3]:
from agent_framework import Executor, WorkflowContext, handler

class UpperCaseExecutor(Executor):
    @handler
    async def to_upper_case(self, text: str, ctx: WorkflowContext[str]) -> None:
        result = text.upper()

        # Persist executor-local state for checkpoints
        prev = await ctx.get_executor_state() or {}
        count = int(prev.get("count", 0)) + 1
        await ctx.set_executor_state({
            "count": count,
            "last_input": text,
            "last_output": result,
        })

        # Send result to next executor
        await ctx.send_message(result)

In [4]:
class ProcessorExecutor(Executor):
    @handler
    async def process(self, text: str, ctx: WorkflowContext[str]) -> None:
        # Write to shared state for cross-executor visibility
        await ctx.set_shared_state("original_input", text)
        await ctx.set_shared_state("processed_output", text.upper())

        await ctx.send_message(text.upper())

In [5]:
# List all checkpoints
all_checkpoints = await checkpoint_storage.list_checkpoints()

# List checkpoints for a specific workflow
workflow_checkpoints = await checkpoint_storage.list_checkpoints(workflow_id="my-workflow")

# Sort by creation time
sorted_checkpoints = sorted(all_checkpoints, key=lambda cp: cp.timestamp)

In [7]:
from agent_framework import RequestInfoExecutor

for checkpoint in all_checkpoints:
    # Get human-readable summary
    summary = RequestInfoExecutor.checkpoint_summary(checkpoint)

    print(f"Checkpoint: {summary.checkpoint_id}")
    print(f"Iteration: {summary.iteration_count}")
    print(f"Status: {summary.status}")
    print(f"Messages: {len(checkpoint.messages)}")
    print(f"Shared State: {checkpoint.shared_state}")
    print(f"Executor States: {list(checkpoint.executor_states.keys())}")

In [8]:
# Resume from a specific checkpoint
async for event in workflow.run_stream_from_checkpoint(
    checkpoint_id="checkpoint-id",
    checkpoint_storage=checkpoint_storage
):
    print(f"Resumed Event: {event}")

    if isinstance(event, WorkflowOutputEvent):
        print(f"Final Result: {event.data}")
        break

NameError: name 'workflow' is not defined